## Imports

In [39]:
import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Data

### Strategies

In [40]:
fundamental_equity = pd.read_csv("../Results/fundamental_equity.csv", index_col=0, parse_dates=True)
macroRS_equity = pd.read_csv("../Results/macroRS_equity.csv", index_col=0, parse_dates=True)

In [41]:
fundamental_equity.rename(columns={"w_GB": "w_UK"}, inplace=True)
fundamental_equity

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_UK,w_JP,w_US
Date,,,,,,,,,,,,,
2000-10-31,-0.014490,-0.014490,-0.017086,0.516370,0.0,0.006621,0.000000,0.000000,0.000000,0.000000,-0.670686,0.166702,0.195352
2000-11-30,0.014938,0.014938,-0.060660,0.564591,0.0,0.004208,0.000000,0.000000,0.000000,0.000000,-0.151213,-0.288488,0.027816
2000-12-31,0.013422,0.013422,0.015919,0.411406,0.0,0.006017,0.000000,0.000000,0.000000,0.000000,0.297918,0.026906,-0.049995
2001-01-31,-0.003768,-0.003768,0.019408,0.091338,0.0,0.006319,0.000000,0.000000,0.000000,0.000000,0.336243,0.153146,-0.018464
2001-02-28,-0.000051,-0.000051,-0.084288,0.238369,0.0,0.003807,0.000000,0.000000,0.000000,0.000000,0.387156,0.003257,-0.292580
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.004282,0.004282,0.043488,0.434028,0.0,0.004008,0.249290,-0.227893,-0.518467,-0.343646,0.240491,-0.325362,0.924918
2025-07-31,0.002353,0.002353,0.013121,1.024281,0.0,0.004409,0.140290,0.089234,-0.381472,-0.220241,0.249742,0.098141,0.097146
2025-08-31,-0.008566,-0.008566,0.026408,1.295210,0.0,0.000000,-0.150868,0.594002,-0.041226,0.074783,-0.376826,-0.340647,0.195866


In [42]:
macroRS_equity

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_AU,w_CH,w_JP,w_UK,w_EM,w_EU
Date,,,,,,,,,,,,,
2002-09-30,-0.066517,-0.066517,-0.109650,0.000000,0.0,0.0014,0.268583,-0.447638,-0.268583,-0.089528,0.089528,0.000000,0.447638
2002-10-31,0.073873,0.073873,0.073988,0.199622,0.0,0.0014,0.251655,-0.419426,-0.083885,-0.251655,0.083885,0.000000,0.419426
2002-11-30,0.017593,0.017593,0.054137,0.558988,0.0,0.0012,0.077098,-0.385488,-0.231293,0.231293,-0.077098,0.000000,0.385488
2002-12-31,-0.018405,-0.018405,-0.048213,0.169611,0.0,0.0011,0.232829,-0.388048,-0.232829,0.077610,-0.077610,0.000000,0.388048
2003-01-31,-0.026167,-0.026167,-0.030204,0.322039,0.0,0.0010,0.231576,-0.385960,-0.077192,-0.231576,0.077192,0.000000,0.385960
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.010496,-0.010496,0.043488,0.280533,0.0,0.0034,0.258529,0.387794,-0.258529,-0.129265,0.129265,-0.387794,0.000000
2025-07-31,0.004953,0.004953,0.013121,0.423485,0.0,0.0034,0.256938,0.385408,0.128469,-0.128469,0.000000,-0.385408,-0.256938
2025-08-31,0.018012,0.018012,0.026408,0.795353,0.0,0.0038,-0.131010,0.393031,0.262021,0.131010,-0.262021,0.000000,-0.393031


### Asset Data

In [43]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    "../Data_Ryan/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

equity_prices

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,1206.821289,NaN,NaN,7.33429,7487.55371,NaN,NaN
1997-10-31,1168.258667,NaN,NaN,6.82642,7084.46436,NaN,NaN
1997-11-30,1207.453491,NaN,NaN,6.34016,7152.55615,NaN,NaN
1997-12-31,1223.840210,NaN,NaN,5.86087,7365.37451,NaN,NaN
1998-01-31,1234.778442,NaN,NaN,6.36481,7690.75049,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,6308.888672,5641.31592,14994.83984,19.64406,12121.12598,14955.800000,6276.48438
2025-07-31,6430.451172,5627.64307,14552.98438,19.41757,12131.93066,15094.600000,6109.94238
2025-08-31,6529.819336,5881.38037,15274.74121,20.75129,12521.77637,15223.700000,6284.96094


In [44]:
equity_returns = equity_prices.pct_change()

### Riskfree Data

In [45]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    "../Data_Ryan/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_86189/2112351923.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Factor Data

In [46]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    "../Data_Ryan/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_86189/3295830964.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745,0.0695
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789,-0.0132
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958,-0.1181
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048,0.0574
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118,0.0618
...,...,...,...,...,...,...,...
2025-05-31,0.0606,-0.0072,-0.0288,0.0129,0.0251,0.0221,0.0256
2025-06-30,0.0486,-0.0002,-0.0160,-0.0320,0.0145,-0.0264,0.0527
2025-07-31,0.0198,-0.0015,-0.0127,-0.0029,-0.0208,-0.0096,0.0184


## Weights

In [47]:
def extract_clean_weights(df):
    # 1) Nur die weight-Spalten nehmen
    weight_cols = [c for c in df.columns if c.startswith("w_")]
    
    # 2) Gewichtstabelle extrahieren
    w = df[weight_cols].copy()
    
    # 3) Prefix "w_" entfernen
    w.columns = [c.replace("w_", "") for c in weight_cols]
    
    return w

# Clean weights for both strategies
fundamental_weights = extract_clean_weights(fundamental_equity)
macroRS_weights = extract_clean_weights(macroRS_equity)


In [48]:
fund, macro = fundamental_weights.align(macroRS_weights, join="inner")
combined_weights = 0.5 * fundamental_weights + 0.5 * macroRS_weights
combined_weights.dropna(how="all", inplace=True)
combined_weights


,AU,CH,EM,EU,JP,UK,US
Date,,,,,,,
2002-09-30,-0.223819,-0.164428,0.000000,0.278387,-0.275530,0.257496,0.224490
2002-10-31,-0.209713,-0.037605,0.000000,0.215924,-0.447455,0.404351,0.224833
2002-11-30,-0.192744,-0.029084,0.000000,0.223082,0.067945,-0.185186,0.095848
2002-12-31,-0.194024,-0.001022,0.000000,0.151480,0.244038,-0.319047,0.295204
2003-01-31,-0.192980,0.037599,0.000000,0.168323,0.058240,-0.107847,0.260366
...,...,...,...,...,...,...,...
2025-06-30,0.318542,-0.243211,-0.453131,-0.171823,-0.227313,0.184878,0.591724
2025-07-31,0.262849,0.108852,-0.383440,-0.238590,-0.015164,0.124871,0.177042
2025-08-31,0.121081,0.428011,-0.020613,-0.159124,-0.104819,-0.319423,0.032428


## Backtest